<a href="https://colab.research.google.com/github/prxrwx/Simulation/blob/main/Queue-simulate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpy

In [ ]:
import simpy
import random
from statistics import mean 
# this allows us to take a mean of a list easily

SimPy in Queuing simulation and store results in list

In [ ]:
# arrivals generator function
def patient_generator_GP(env, wl_inter, mean_regist, mean_consult, mean_booktest, receptionist1, 
                         gp, receptionist2):
  
  while True:
    # create instance of activity generator
    wp = activity_generator_GP(env, mean_regist, mean_consult, mean_booktest,
                          receptionist1, gp, receptionist2)

    # run the activity generator for patient p_id
    env.process(wp)

    # sample time until next patient
    t = random.expovariate(1.0 / wl_inter)

    # freeze until that time has passed
    yield env.timeout(t)

In [ ]:
def activity_generator_GP(env, mean_regist, mean_consult, mean_booktest,
                          receptionist1, gp, receptionist2):
  global list_of_queue_times_registration # list for storing results, declare to be global
  global list_of_queue_times_consultation
  global list_of_queue_times_booktest

  time_enqueue_of_registration = env.now
  
  # request a receptionist1
  with receptionist1.request() as req:
    # freeze until the request can be met
    yield req

    time_dequeue_of_registration = env.now
    time_in_queue_for_registration = (time_dequeue_of_registration - time_enqueue_of_registration)
    
    list_of_queue_times_registration.append(time_in_queue_for_registration) # add time in Q for receptionist1 to list

    sampled_registration_time = random.expovariate(1.0/mean_regist)  # sample the time spent in registration

    # freeze until that time has passed
    yield env.timeout(sampled_registration_time)

# The above with statement, assign with the receptionist. Now start for queuing of the nurse.
  time_enqueue_of_consultation = env.now

  # request a GP
  with gp.request() as req:
    # freeze until the request can be met
    yield req

    # calculate time patient was queuing
    time_dequeue_of_consultation = env.now
    time_in_queue_for_consultation = (time_dequeue_of_consultation - time_enqueue_of_consultation)

    list_of_queue_times_consultation.append(time_in_queue_for_consultation) # add time in Q for GP to list
    
    sampled_consultation_time = random.expovariate(1.0/mean_consult)

    # freeze until that time has passed
    yield env.timeout(sampled_consultation_time)     

    # Branching path for Book Test or Sink          
    booktest_branch = random.uniform(0,1)             
    if booktest_branch < 0.25:
      time_enqueue_of_booktest = env.now

      # request a receptionist2
      with receptionist2.request() as req:
        # freeze until the request can be met
        yield req

        time_dequeue_of_booktest = env.now
        time_in_queue_for_booktest = (time_dequeue_of_booktest - time_enqueue_of_booktest)

        list_of_queue_times_booktest.append(time_in_queue_for_booktest) # add time in Q for receptionist2 to list

        sampled_booktest_time = random.expovariate(1.0/mean_booktest)
        # freeze until that time has elapsed
        yield env.timeout(sampled_booktest_time)
    else:
      yield env.timeout(sampled_consultation_time)

In [ ]:
# Set up simulation environment
env = simpy.Environment()
# Set up resources
receptionist1 = simpy.Resource(env, capacity = 1)
receptionist2 = simpy.Resource(env, capacity = 1)
gp = simpy.Resource(env, capacity = 2)
# Set up parameter values
wl_inter = 5
mean_regist = 2
mean_consult = 3
mean_booktest = 5

# Set up a list to store queuing time 
list_of_queue_times_registration = []
list_of_queue_times_consultation =[]
list_of_queue_times_booktest =[]
#Start the arrivals generator
env.process(patient_generator_GP(env, wl_inter, mean_regist, mean_consult, mean_booktest, 
                                 receptionist1, gp, receptionist2))
# run the simulation
env.run(until = 480)

# calculate and print mean queuing time
mean_queue_time_registration = mean(list_of_queue_times_registration)
print("Mean queuing time of registration (mins) : ", mean_queue_time_registration)
mean_queue_time_consultation = mean(list_of_queue_times_consultation)
print("Mean queuing time of consultation (mins) : ", mean_queue_time_consultation)
mean_queue_time_booktest = mean(list_of_queue_times_booktest)
print("Mean queuing time of book test (mins) : ", mean_queue_time_booktest)

Mean queuing time of registration (mins) :  1.189891820672492
Mean queuing time of consultation (mins) :  1.8502772610679206
Mean queuing time of book test (mins) :  0.4715160608702119
